# Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import MACCSkeys
from copy import deepcopy
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from padelpy import from_sdf
import shap
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/KRAS_work_from_insilico.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pIC50_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  452 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

452

# Standardization SDF file for work set

In [5]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  452 molecules


In [6]:
moldf_ws=pd.DataFrame(moldf_ws, columns=['Mol'])
moldf_ws

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
1,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
2,<rdkit.Chem.rdchem.Mol object at 0x0000024D33D...
3,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
4,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
...,...
447,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
448,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
449,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
450,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...


# Data entry and curation test set

In [7]:
uploaded_file_ts="datasets/KRAS_test_from_insilico.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pIC50_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  114 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [8]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [9]:
len(y_ts)

114

# Standardization SDF file for test set

In [10]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  114 molecules


In [11]:
moldf_ts=pd.DataFrame(moldf_ts, columns=['Mol'])
moldf_ts

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
1,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
2,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
3,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
4,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
...,...
109,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
110,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
111,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...
112,<rdkit.Chem.rdchem.Mol object at 0x0000024D33E...


# Calculation ECFP4 for work set

In [13]:
import warnings
from rdkit import RDLogger

# Suppress RDKit warnings (including deprecation warnings)
RDLogger.DisableLog('rdApp.*')

def calcfp_ECFP4(mol, funcFPInfo=dict(radius=2, nBits=1024, useFeatures=False, useChirality=False)):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, **funcFPInfo)
    fp = pd.Series(np.asarray(fp))
    fp = fp.add_prefix('Bit_')
    return fp

In [14]:
# Training set
desc_ws_ECFP4 = moldf_ws.Mol.apply(calcfp_ECFP4)
desc_ws_ECFP4

,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,...,Bit_1014,Bit_1015,Bit_1016,Bit_1017,Bit_1018,Bit_1019,Bit_1020,Bit_1021,Bit_1022,Bit_1023
0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
448,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
449,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
450,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


# Calculation Topological Path-Based Fingerprints (RDKit) for work set

In [19]:
from rdkit.Chem import rdFingerprintGenerator
def calcfp_topPB(mol, funcFPInfo=dict(minPath=1, maxPath=7, fpSize=2048, useHs=True,
                                branchedPaths=True, useBondOrder=True)):
    """
    RDKit Topological Fingerprint (path-based)
    Returns numpy array for easier DataFrame conversion
    """
    fpgen = rdFingerprintGenerator.GetRDKitFPGenerator(**funcFPInfo)
    fp_topPB = fpgen.GetFingerprint(mol)
    # Return numpy array directly - no Series conversion
    return np.asarray(fp_topPB)

In [20]:
desc_ws_topPB = moldf_ws.Mol.apply(calcfp_topPB)
# Convert to regular DataFrame (calcfp returns numpy arrays)
desc_ws_topPB = pd.DataFrame(list(desc_ws_topPB), index=moldf_ws.index)
desc_ws_topPB

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1,1,0,1,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
1,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
2,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
3,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
4,1,1,0,1,1,1,1,1,1,0,...,1,0,0,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,1,1,0,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1
448,1,1,0,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1
449,1,1,0,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1
450,1,1,0,1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,1,1


# Calculation RDKit_2D descriptors for work set

In [21]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
header = calc.GetDescriptorNames()

In [22]:
descr_tr= []
for m in moldf_ws.Mol:
    descr_tr.append(calc.CalcDescriptors(m))
x_tr = np.asarray(descr_tr)

In [23]:
df_RDKit_2D = pd.DataFrame(x_tr,columns=header)

In [24]:
df_RDKit_2D.isna().mean().sort_values(ascending=False).head(15)

MaxAbsEStateIndex          0.0
fr_NH2                     0.0
fr_Ar_NH                   0.0
fr_Ar_OH                   0.0
fr_COO                     0.0
fr_COO2                    0.0
fr_C_O                     0.0
fr_C_O_noCOO               0.0
fr_C_S                     0.0
fr_HOCCN                   0.0
fr_Imine                   0.0
fr_NH0                     0.0
fr_NH1                     0.0
fr_N_O                     0.0
NumAliphaticCarbocycles    0.0
dtype: float64

In [25]:
x_tr= df_RDKit_2D.to_numpy ()

In [26]:
# Data Standardization
from sklearn.preprocessing import StandardScaler
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)


In [29]:
x_tr.shape

(452, 217)

In [31]:
x_tr_RDKit_2D = np.array(x_tr, dtype=np.float32)

# Calculation ECFP4 for test set

In [32]:
desc_ts_ECFP4 = moldf_ts.Mol.apply(calcfp)
desc_ts_ECFP4

,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,...,Bit_1014,Bit_1015,Bit_1016,Bit_1017,Bit_1018,Bit_1019,Bit_1020,Bit_1021,Bit_1022,Bit_1023
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
110,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
111,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
112,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [33]:
y_ts = np.array(y_ts, dtype=np.float32)
len(y_ts)

114

# Calculation Topological Path-Based Fingerprints (RDKit) for test set¶

In [35]:
desc_ts_topPB = moldf_ts.Mol.apply(calcfp_topPB)
desc_ts_topPB = pd.DataFrame(list(desc_ts_topPB), index=desc_ts_topPB.index)
desc_ts_topPB


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1,1,0,0,0,1,1,0,1,0,...,1,0,0,1,1,0,1,1,1,1
1,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
2,1,1,0,1,0,1,1,1,1,0,...,1,0,1,1,1,0,0,1,1,1
3,1,1,0,0,1,1,1,1,1,0,...,0,1,0,1,1,0,0,1,1,1
4,1,1,1,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,1,1,1,1,0,1,1,0,0,0,...,1,0,0,1,1,0,0,1,1,1
110,1,1,0,1,0,1,1,0,1,0,...,1,0,0,1,1,0,1,1,1,1
111,1,1,0,1,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1
112,1,1,0,0,0,1,1,0,0,0,...,1,0,0,1,1,0,1,1,1,1


# Calculation RDKit_2D descriptors for test set

In [36]:
descr_ts__RDKit_2D = []
for m in moldf_ts.Mol:
    descr_ts__RDKit_2D.append(calc.CalcDescriptors(m))
x_ts__RDKit_2D = np.asarray(descr_ts__RDKit_2D)

In [37]:
x_ts__RDKit_2D.shape

(114, 217)

In [38]:
df_RDKit_2D_ts = pd.DataFrame(x_ts__RDKit_2D,columns=header)
df_RDKit_2D_ts=df_RDKit_2D_ts.dropna(axis=1)
df_RDKit_2D_ts

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,16.546704,16.546704,0.114958,-0.502508,0.313304,22.150000,556.045,528.829,555.194964,202.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16.265427,16.265427,0.124335,-0.521930,0.393832,20.277778,507.997,480.781,507.183731,186.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,16.372442,16.372442,0.105215,-0.848130,0.502232,28.538462,535.643,500.363,535.287115,206.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.416565,15.416565,0.195899,-0.945490,0.512666,24.948718,536.627,502.355,536.271131,206.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.356441,16.356441,0.136609,-0.519813,0.336462,18.947368,530.007,504.807,529.179314,192.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,16.281300,16.281300,0.018730,-5.022994,0.416075,23.756098,574.579,544.339,574.231552,218.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110,16.544045,16.544045,0.008592,-0.901026,0.353085,27.309524,576.623,545.375,576.246059,218.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,16.988743,16.988743,0.026715,-0.683690,0.249421,24.660000,687.792,644.448,687.334459,264.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,16.470194,16.470194,0.005035,-0.593601,0.369208,23.256410,549.050,518.810,548.210280,202.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
x_ts_2D_ts= df_RDKit_2D_ts.to_numpy ()

In [40]:
x_ts_2D_ts = scale.transform(x_ts_2D_ts)

In [41]:
x_ts_2D_ts.shape

(114, 217)

# load the models from disk

In [42]:
ECFP4_model = pickle.load(open('Models/ECFP4/CatBoost_MF.pkl', 'rb'))

In [43]:
topPB_model = pickle.load(open('Models/TopologicalFingerprint/CatBoost_TF.pkl', 'rb'))

In [44]:
RDKit_2D_model = pickle.load(open('Models/RDKiT/CatBoost_RDKiT.pkl', 'rb'))

# Prediction for CV

In [47]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [48]:
params={'verbose': False}

## ECFP4

In [62]:
%%time
y_pred_CV_ECFP4 = cross_val_predict(ECFP4_model, desc_ws_ECFP4, y_tr, cv=cv, params=params)

CPU times: total: 22min 53s
Wall time: 2min 6s


In [63]:
Q2_CV_ECFP4 = round(r2_score(y_tr, y_pred_CV_ECFP4), 2)
Q2_CV_ECFP4

0.65

In [73]:
y_pred_CV_ECFP4 = np.asarray(y_pred_CV_ECFP4, dtype=float)

In [74]:
y_tr = np.array(y_tr, dtype=float)

In [75]:
RMSE_CV_ECFP4=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_ECFP4)), 2)
RMSE_CV_ECFP4

0.73

## topPB

In [76]:
%%time
y_pred_CV_topPB = cross_val_predict(topPB_model, desc_ws_topPB, y_tr, cv=cv, params=params)

CPU times: total: 11min 46s
Wall time: 1min


In [77]:
Q2_CV_topPB = round(r2_score(y_tr, y_pred_CV_topPB), 2)
Q2_CV_topPB

0.6

In [78]:
y_pred_CV_topPB = np.asarray(y_pred_CV_topPB, dtype=float)

In [79]:
RMSE_CV_topPB=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_topPB)), 2)
RMSE_CV_topPB

0.77

## RDKit_2D

In [82]:
%%time
y_pred_CV_RDKit_2D = cross_val_predict(RDKit_2D_model, x_tr_RDKit_2D, y_tr, cv=cv, params=params)

CPU times: total: 4min 40s
Wall time: 19.1 s


In [83]:
Q2_CV_RDKit_2D = round(r2_score(y_tr, y_pred_CV_RDKit_2D), 2)
Q2_CV_RDKit_2D

0.64

In [86]:
y_pred_CV_RDKit_2D = np.asarray(y_pred_CV_RDKit_2D, dtype=float)

In [88]:
RMSE_CV__RDKit_2D=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_RDKit_2D)), 2)
RMSE_CV__RDKit_2D

0.73

In [90]:
y_pred_con=(y_pred_CV_ECFP4+y_pred_CV_topPB+y_pred_CV_RDKit_2D)/3

In [91]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.68

In [92]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_con)),2)
RMSE_CV

0.69

# Prediction for test set's molecules

In [93]:
y_pred_ECFP4 = ECFP4_model.predict(desc_ts_ECFP4)

In [94]:
y_pred_topPB = topPB_model.predict(desc_ts_topPB)

In [95]:
y_pred_RDKit_2D = RDKit_2D_model.predict(x_ts_2D_ts)

In [171]:
# x_ts = np.array(x_ts, dtype=np.float32)
# y_ts = np.array(y_ts, dtype=np.float32)

In [96]:
y_pred_con=(y_pred_ECFP4+y_pred_topPB+y_pred_RDKit_2D)/3

In [97]:
Q2_TS = round(r2_score(y_ts, y_pred_con), 2)
Q2_TS

0.71

In [176]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_con)), 2)
RMSE_TS

0.69

# Estimating applicability domain. Method - Euclidian distances, K=1

In [115]:
neighbors_k= pairwise_distances(desc_ws_ECFP4, n_jobs=-1)
neighbors_k.sort(0)

In [116]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.872983,4.898979,3.605551,3.464102,2.645751,4.000000,3.464102,4.358899,3.000000,5.196152,...,3.741657,3.316625,3.162278,3.605551,3.316625,3.162278,3.605551,3.162278,3.162278,3.316625
2,4.000000,4.898979,3.741657,4.123106,3.000000,4.123106,4.123106,5.385165,4.898979,6.480741,...,3.741657,3.316625,3.464102,3.741657,3.316625,3.162278,5.000000,3.464102,3.162278,4.472136
3,4.472136,4.898979,3.741657,4.242641,3.872983,4.242641,4.242641,5.477226,5.099020,6.633250,...,3.741657,3.741657,3.605551,3.741657,3.464102,3.464102,5.196152,3.605551,3.162278,4.795832
4,4.472136,5.000000,3.872983,4.795832,4.000000,4.358899,4.242641,5.477226,5.196152,6.633250,...,3.872983,3.872983,3.741657,4.000000,3.464102,3.605551,5.196152,4.123106,3.464102,4.898979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,9.433981,10.049876,9.539392,9.797959,9.746794,9.949874,9.797959,9.643651,10.344080,9.746794,...,9.643651,9.380832,9.380832,10.000000,9.273618,9.746794,9.949874,10.049876,9.433981,10.630146
448,9.486833,10.049876,9.643651,9.848858,9.797959,10.049876,9.848858,9.695360,10.344080,9.797959,...,9.643651,9.433981,9.433981,10.000000,9.327379,9.848858,10.099505,10.099505,9.433981,10.677078
449,9.486833,10.148892,9.643651,9.848858,9.797959,10.049876,9.848858,9.695360,10.440307,9.848858,...,9.695360,9.539392,9.486833,10.099505,9.486833,9.899495,10.148892,10.148892,9.486833,10.770330
450,9.591663,10.344080,9.746794,9.848858,9.848858,10.049876,9.949874,9.746794,10.440307,9.848858,...,9.797959,9.643651,9.539392,10.099505,9.591663,9.899495,10.198039,10.246951,9.591663,10.862780


In [117]:
similarity= neighbors_k

In [118]:
Dmean=np.mean(similarity[1,:])

In [119]:
round(Dmean, 2)

3.5

In [120]:
std=np.std(similarity[1,:])

In [121]:
round(std, 2)

0.82

In [122]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

3.91


In [123]:
x_tr.shape

(452, 217)

In [124]:
neighbors_k_ts= pairwise_distances(desc_ws_ECFP4,Y=desc_ts_ECFP4, n_jobs=-1)
neighbors_k_ts.sort(0)

In [125]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,2.828427,3.000000,3.162278,4.582576,4.000000,5.099020,3.000000,3.741657,1.732051,3.162278,...,4.000000,3.316625,3.162278,3.741657,3.162278,3.162278,3.316625,3.872983,3.316625,3.000000
1,5.291503,3.605551,3.605551,4.582576,4.123106,5.291503,3.000000,3.872983,3.162278,3.316625,...,4.582576,4.795832,3.316625,3.872983,3.464102,3.605551,3.464102,3.872983,3.464102,3.741657
2,5.291503,4.000000,4.123106,5.000000,4.123106,5.291503,3.162278,4.123106,3.741657,3.316625,...,5.477226,5.385165,3.464102,4.000000,3.464102,4.000000,3.464102,4.898979,3.741657,3.741657
3,5.477226,4.123106,4.242641,5.000000,4.242641,5.385165,3.162278,4.242641,3.741657,3.464102,...,5.567764,5.656854,3.741657,4.000000,3.464102,4.123106,3.464102,5.196152,4.358899,3.872983
4,5.477226,4.123106,4.242641,5.000000,4.358899,5.385165,3.162278,4.358899,4.123106,3.605551,...,6.082763,5.744563,4.123106,4.000000,3.741657,4.358899,3.464102,5.291503,4.472136,3.872983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,10.246951,9.695360,9.797959,9.539392,9.848858,10.148892,9.165151,9.643651,10.198039,9.219544,...,10.535654,9.797959,9.695360,9.746794,9.273618,9.591663,9.695360,10.198039,9.746794,9.273618
448,10.246951,9.797959,9.797959,9.591663,9.848858,10.148892,9.165151,9.746794,10.198039,9.219544,...,10.583005,9.797959,9.695360,9.797959,9.380832,9.643651,9.746794,10.246951,9.746794,9.273618
449,10.344080,9.797959,9.899495,9.591663,9.848858,10.246951,9.327379,9.746794,10.295630,9.219544,...,10.583005,9.797959,9.848858,9.848858,9.433981,9.695360,9.797959,10.344080,9.848858,9.433981
450,10.344080,9.797959,9.949874,9.695360,10.049876,10.344080,9.380832,10.049876,10.344080,9.273618,...,10.630146,9.848858,9.949874,9.899495,9.539392,9.746794,9.899495,10.392305,9.949874,9.486833


In [126]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.828 3.    3.162 4.583 4.    5.099 3.    3.742 1.732 3.162 5.292 4.796
 4.    4.    3.742 3.742 5.099 5.196 4.    3.317 5.385 3.606 4.243 3.742
 3.162 3.162 4.796 3.742 4.123 3.464 3.    3.    3.162 3.464 4.796 3.162
 2.236 3.464 5.831 3.162 2.236 3.606 3.606 2.828 2.449 3.    2.236 3.
 3.162 3.606 3.162 3.317 3.162 3.162 3.873 4.243 4.359 3.464 5.477 3.606
 2.236 3.317 3.464 3.317 3.    2.828 3.    3.    2.236 3.162 3.606 3.742
 3.317 3.162 3.162 3.    3.317 3.    3.317 4.472 4.    5.385 3.    2.646
 3.742 3.    3.317 4.243 4.69  3.    3.162 3.317 3.606 3.464 2.646 3.162
 3.162 2.828 3.317 1.732 3.    3.    3.    3.742 4.    3.317 3.162 3.742
 3.162 3.162 3.317 3.873 3.317 3.   ]


In [127]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True False False False  True  True  True  True False False
 False False  True  True False False False  True False  True False  True
  True  True False  True False  True  True  True  True  True False  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False False  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False False False  True  True
  True  True  True False False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True]


In [128]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7807017543859649


In [129]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   1   2   6   7   8   9  14  15  19  21  23  24  25  27  29  30  31
  32  33  35  36  37  39  40  41  42  43  44  45  46  47  48  49  50  51
  52  53  54  57  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  82  83  84  85  86  89  90  91  92  93  94  95
  96  97  98  99 100 101 102 103 105 106 107 108 109 110 111 112 113]


In [130]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [131]:
y_pred_con_ad=list(y_pred_con)

In [132]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [133]:
len(y_pred_con_ad)

89

In [134]:
y_ts_ad=list(y_ts)

In [135]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [136]:
len(y_ts_ad)

89

In [137]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.7

In [139]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.66